In [ ]:
# Importar librerias
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/ReconocimientoPatrones/train.csv')
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# Funcion para dividir age en 4 categorias
def age_class(x):
    if np.double(x)>= 0 and np.double(x) < 18:
        x = 1.0;
    elif np.double(x) >=18 and np.double(x)< 25:
        x = 2.0;
    elif np.double(x) >=25 and np.double(x) < 35:
        x = 3.0;
    else:
        x = 4.0;
    return x;

In [ ]:
# Se aplica la funcion age_class para que ahora las edades esten agrpupadas en
# 4 categorias
data.Age = data['Age'].apply(age_class)

In [ ]:
# Male y female se transforman a 1 y 0
data.Sex = data['Sex'].map({'female':0,'male':1}).astype(int)

In [ ]:
X = data[['Pclass', 'Sex','Age','SibSp','Parch']].values
y = data['Survived'].values

In [ ]:
y_train = y[:600] # Training
y_val = y[600:] # Testing

X_train = X[:600] # Training
X_val = X[600:] # Testing

In [ ]:
# Calculo de distancias
def eucledian(x1,x2):
    distance = np.sqrt(np.sum(pow(x1-x2,2)))
    return distance

In [ ]:
# Import de mode para obtener la moda
# Es decir el valor que más se repite entre las k's
from scipy.stats import mode

#Function to calculate KNN
def KNearestNeighbors(x_train, y_train , x_val, y_val, k):
    accuracy = 0
    error = 0
    index = 0

    # Itera en los elementos a clasificar (testing)
    for i in range(len(x_val)):

        # Aqui se almacenaran todas las distancias
        distances = []

        # Itera en los elementos de entrenamiento
        for j in range(len(x_train)):
            # Calcula la distancia entre x_train y x_val
            distance = eucledian(x_train[j] , x_val[i])
            # Se añaden las distancias calculadas a distances
            distances.append(distance) 
         
        # Se ordenan las distancias y se almacenan los indices de los primeros
        # k (numero de vecinos)
        # En lugar de las distancias se almacena el valor del indice
        idxs = np.argsort(distances)[:k]
         
        # Se almacena en nearest cada elemento de los indices en orderd_dists
        # los elementos en ese caso son 0 o 1
        neighbors = y_train[idxs]
         
        # Se calcula la moda para obtener el elemento que más se repite (0 o 1)
        # Este sera nuestro resultado para la prediccion
        result = mode(neighbors)
        result = result.mode[0]

        if y_val[index] == result:
          accuracy += 1
        else:
          error += 1
        index += 1
        
    accuracy_score = round(accuracy/y_val.shape[0], 6)
    print('number of successes:', accuracy)
    print('number of failures:', error)
    print('accuracy score: %f'%accuracy_score)

In [ ]:
# Resultado de la prediccion de X_val con 4 vecinos
KNearestNeighbors(X_train,y_train,X_val,y_val,4)

<ipython-input-9-84898b97ab2a>:35: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  result = mode(neighbors)


number of successes: 239
number of failures: 52
accuracy score: 0.821306


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_val)

print('Accuracy score:',accuracy_score(y_val, y_pred))

Accuracy score: 0.8075601374570447
